In [1]:
import pandas as pd

### Link to the dataset

https://www.kaggle.com/datasets/wordsforthewise/lending-club/data?select=rejected_2007_to_2018Q4.csv.gz

In [2]:
#rejected_loan_data = pd.read_csv('data/rejected_2007_to_2018Q4.csv.gz')
loan_data = pd.read_csv('data/accepted_2007_to_2018Q4.csv.gz')

C:\Users\abarg\AppData\Local\Temp\ipykernel_6664\37491141.py:2: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_data = pd.read_csv('data/accepted_2007_to_2018Q4.csv.gz')


In [3]:
list(loan_data.columns)

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'last_fico_range_high',
 'last_fico_range_low',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 '

In [4]:
loan_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


`issue_d` Month and year when the loan was issued

`loan_status` Current status of the loan

In [5]:
loan_data['loan_status'].value_counts()

loan_status
Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: count, dtype: int64

In [6]:
loan_data[loan_data['loan_status'].isnull()]

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
421095,Total amount funded in policy code 1: 6417608175,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
421096,Total amount funded in policy code 2: 1944088810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528961,Total amount funded in policy code 1: 1741781700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
528962,Total amount funded in policy code 2: 564202131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
651664,Total amount funded in policy code 1: 1791201400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
651665,Total amount funded in policy code 2: 651669342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
749520,Total amount funded in policy code 1: 1443412975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
749521,Total amount funded in policy code 2: 511988838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
877716,Total amount funded in policy code 1: 2063142975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
877717,Total amount funded in policy code 2: 823319310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
loan_data[loan_data['loan_status'].isnull()].index == loan_data[loan_data['issue_d'].isnull()].index

array([], dtype=bool)

In [8]:
# Drop unnecessary id columns
loan_data = loan_data.drop(columns=['id', 'member_id'])

# Drop null values in issue_d (and target) column
loan_data = loan_data[loan_data['issue_d'].notnull()]

In [10]:
loan_data['issue_d'].head()

0    Dec-2015
1    Dec-2015
2    Dec-2015
3    Dec-2015
4    Dec-2015
Name: issue_d, dtype: object

In [ ]:
#loan_data = loan_data.astype({'issue_d': 'datetime64[ns]'})
#loan_data[['issue_month', 'issue_year']] = loan_data['issue_d'].str.split('-', expand=True)


In [ ]:
# Add year column from issue date
loan_data['issue_year'] = loan_data['issue_d'].str[-4:].astype(int)

In [14]:
loan_data[['issue_d', 'issue_year']].sample(10)

,issue_d,issue_year
2259658,Oct-2016,2016
341657,Mar-2015,2015
582842,Aug-2017,2017
786008,Sep-2018,2018
1011637,Mar-2016,2016
587387,Aug-2017,2017
478447,Feb-2018,2018
100099,Oct-2015,2015
1631218,Feb-2011,2011
1478067,Oct-2018,2018


In [ ]:
# # Convert data types
# rejected_loan_data = rejected_loan_data.astype(
#     {
#         'Application Date': 'datetime64[ns]',
#         'Loan Title': 'str',
#     }
# )

In [ ]:
#rejected_loan_data['Year'] = rejected_loan_data['Application Date'].str[:4].astype(int)

In [ ]:
# # Add year column from application date
# rejected_loan_data['Year'] = rejected_loan_data['Application Date'].dt.year

# # Convert Debt-To-Income Ratio column from percentage string to float
# rejected_loan_data['Debt-To-Income Ratio'] = rejected_loan_data['Debt-To-Income Ratio'].str[:-1].astype(float) / 100